**Loading full dataset**

In [0]:
df_oct = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv", header=True, inferSchema=True)
df_nov = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Nov.csv", header=True, inferSchema=True)
df_oct.printSchema()
df_nov.printSchema()
print(f'Total number of rows in October: {df_oct.count()}')
print(f'Total number of rows in November: {df_nov.count()}')

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)

root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)

Total number of rows in October: 42448764
Total number of rows in November: 67501979


**Appended both month data**

In [0]:
from pyspark.sql.functions import year, month, col
both = df_oct.union(df_nov)

# Created Year and month column
df_both = both.withColumn("year", year(col("event_time"))).withColumn("month", month(col("event_time")))

df_both.printSchema()


root
 |-- event_time: timestamp (nullable = true)
 |-- event_type: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- category_id: long (nullable = true)
 |-- category_code: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- price: double (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- user_session: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)



In [0]:
df_both.select("year", "month").distinct().show()
print(f'Total number of rows after combining both months: {df_both.count()}')

+----+-----+
|year|month|
+----+-----+
|2019|   10|
|2019|   11|
+----+-----+

Total number of rows after combining both months: 109950743


**Creating Product dimenstion table**

In [0]:
product_dim = df_both.select(
    "product_id",
    "brand",
    "category_code"
).distinct()

**Creating Fact table for only purchased**

In [0]:
purchase_fact = df_both.filter(df_both.event_type == "purchase")


**Joins**

In [0]:
# Joining purchase_fact with product_dim
fact_prod_join = purchase_fact.join(
    product_dim,
    on="product_id",
    how="inner"
).select(
    purchase_fact["*"],
    product_dim["category_code"]
)



In [0]:
fact_prod_join.show(5)

+-------------------+----------+----------+-------------------+--------------------+-------+------+---------+--------------------+----+-----+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code|  brand| price|  user_id|        user_session|year|month|       category_code|
+-------------------+----------+----------+-------------------+--------------------+-------+------+---------+--------------------+----+-----+--------------------+
|2019-10-01 00:02:14|  purchase|   1004856|2053013555631882655|electronics.smart...|samsung|130.76|543272936|8187d148-3c41-46d...|2019|   10|electronics.smart...|
|2019-10-01 00:04:37|  purchase|   1002532|2053013555631882655|electronics.smart...|  apple|642.69|551377651|3c80f0d6-e9ec-418...|2019|   10|electronics.smart...|
|2019-10-01 00:06:02|  purchase|   5100816|2053013553375346967|                NULL| xiaomi| 29.51|514591159|0e5dfc4b-2a55-43e...|2019|   10|                NULL|
|2019-10-01 00:07:07| 

**Running Total**

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import sum

fact_running = fact_prod_join.withColumn(
    "running_revenue",
    sum("price").over(Window.partitionBy("brand").orderBy("event_time").
                      rowsBetween(Window.unboundedPreceding, Window.currentRow))
)


**Viewing the Running Total**

In [0]:
fact_running.select(
    "brand",
    "event_time",
    "price",
    "running_revenue"
).show(10)


+-----+-------------------+------+---------------+
|brand|         event_time| price|running_revenue|
+-----+-------------------+------+---------------+
| NULL|2019-10-01 02:19:59|153.16|         153.16|
| NULL|2019-10-01 02:20:28| 91.12|         244.28|
| NULL|2019-10-01 02:21:45| 60.49|         304.77|
| NULL|2019-10-01 02:23:03|120.47|         425.24|
| NULL|2019-10-01 02:26:02| 33.46|          458.7|
| NULL|2019-10-01 02:30:49| 14.16|         472.86|
| NULL|2019-10-01 02:34:27|226.78|         699.64|
| NULL|2019-10-01 02:35:33|431.67|        1131.31|
| NULL|2019-10-01 02:35:33|431.67|        1562.98|
| NULL|2019-10-01 02:35:34|189.71|        1752.69|
+-----+-------------------+------+---------------+
only showing top 10 rows


In [0]:
Rev_by_brand = (
    fact_prod_join
    .groupBy("brand")
    .agg(sum("price").alias("revenue"))
    .orderBy("revenue", ascending=False)
)

In [0]:
from pyspark.sql.functions import dense_rank

rank_window = Window.orderBy(Rev_by_brand.revenue.desc())

Rev_by_brand.withColumn(
    "Rank",
    dense_rank().over(rank_window)
).show()


/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


+--------+--------------------+----+
|   brand|             revenue|Rank|
+--------+--------------------+----+
|   apple|2.4124841642999887E8|   1|
| samsung|1.1331158595999838E8|   2|
|  xiaomi|2.4322094730000027E7|   3|
|    NULL|2.0771621959999997E7|   4|
|  huawei|   9781475.429999994|   5|
|      lg|          9269535.73|   6|
|    oppo|   8069405.960000003|   7|
|    acer|   7531656.740000002|   8|
| lucente|   6944021.140000003|   9|
|    sony|   6443957.189999998|  10|
|  lenovo|          5461337.68|  11|
|   bosch|   3167678.170000001|  12|
|    asus|   3157054.659999999|  13|
|   artel|   2725263.849999999|  14|
|      hp|   2641555.290000001|  15|
| indesit|  2580108.6100000013|  16|
|   haier|  2019021.3399999994|  17|
|    beko|  1869392.5800000008|  18|
|dauscher|  1522904.3699999994|  19|
|cordiant|  1323042.6000000003|  20|
+--------+--------------------+----+
only showing top 20 rows
